In [1]:
#Import relevant modules
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

In [2]:
# get coming race day from schedule csv
df = pd.read_csv('../data/schedule/schedule_s2021.csv')
df.columns = ['year','month','day']
df['next_race_day']=pd.to_datetime(df)
min_idx=df[df['next_race_day']>pd.Timestamp('today')]['next_race_day'].idxmin()
next_race_day=df.loc[min_idx,'next_race_day']
coming_race_day=str(next_race_day.year)+'-'+str(next_race_day.month)+'-'+str(next_race_day.day)
coming_race_day

'2020-11-8'

In [3]:
driver = webdriver.Chrome('/Users/Hei/Applications/chromedriver')

In [4]:
coming_race_day
suffix_raceno='1' #hardcode for test, default =1
URL_coming_race = 'https://bet.hkjc.com/racing/pages/odds_wp.aspx?lang=en&date='+coming_race_day+'&raceno='+suffix_raceno
URL_coming_race

'https://bet.hkjc.com/racing/pages/odds_wp.aspx?lang=en&date=2020-11-8&raceno=1'

In [5]:
# Initialize soup for getting number of race
driver.get(URL_coming_race)
time.sleep(5)
subhtml = driver.page_source
soup = BeautifulSoup(subhtml, 'html.parser')

# Initialize dataframe
df_race = pd.DataFrame(columns=['season','race','horse_no','horse','jockey','trainer','class','track','location',\
                                'actual_weight','draw','distance_m','win_odds','date'])

# Get location
location=soup.find('div',{'class':'mtgInfoDV'}).text.split(',')[2].strip()

In [6]:
def get_no_of_race():
    racebg = soup.find(class_="racebg").contents
    count_race=0
    for i in range(len(racebg)):
        if racebg[i] !='\n' and len(racebg[i].attrs)==4 and 'id' in racebg[i].attrs.keys():
            count_race+=1
    return count_race

def get_race_info():
    pattern=[p.text for p in soup.find_all('strong')]
    line=' '.join(pattern)
    match_str = re.search(r'Race\s(\d+)', line)
    race=match_str.group(1)
    race_info=soup.find('span',{'class':'content'}).text.split(',')
    date=race_info[1].strip()
    race_class=race_info[3].strip()
    turf=race_info[4].strip()
    course=race_info[5].strip()
    track=turf+' - '+course
    distance=race_info[6].strip()[:-1]
    return race,date,race_class,track,distance

def write_table():
    race_no,date,race_class,track,distance=get_race_info()
    dict_table={'season':[],'race':[],'horse_no':[],'horse':[],'jockey':[],'trainer':[],'class':[],'track':[],\
                'location':[],'actual_weight':[],'draw':[],'distance_m':[],'win_odds':[],'date':[]}
    table = soup.find_all(id='horseTable')[0].tbody.find_all('td')[:-6]
    for i in range(len(table)):
        if i %10==0:
            dict_table['season'].append('2020/2021')
            dict_table['race'].append(race_no)
            dict_table['class'].append(race_class)
            dict_table['track'].append(track)
            dict_table['location'].append(location)
            dict_table['distance_m'].append(distance)
            dict_table['date'].append(date)
            dict_table['horse_no'].append(table[i].text)
        elif i%10==2:
            dict_table['horse'].append(table[i].text)
        elif i%10==3:
            dict_table['draw'].append(table[i].text)
        elif i%10==4:
            dict_table['actual_weight'].append(table[i].text)
        elif i%10==5:
            dict_table['jockey'].append(table[i].text)
        elif i%10==6:
            dict_table['trainer'].append(table[i].text)
        elif i%10==7:
            dict_table['win_odds'].append(table[i].text)
    df=pd.DataFrame(dict_table)
    return df


In [7]:
for i in range(get_no_of_race()):
    # get soup for each race
    URL_coming_race = 'https://bet.hkjc.com/racing/pages/odds_wp.aspx?lang=en&date='+coming_race_day+'&raceno='+str(i+1)
    
    driver.get(URL_coming_race)
    time.sleep(5)
    subhtml = driver.page_source
    soup = BeautifulSoup(subhtml, 'html.parser')
    
    #get horse inforamtion for each race
    table = soup.find_all(id='horseTable')[0].tbody.find_all('td')[:-6]
    df_race=pd.concat([df_race,write_table()])


,season,race,horse_no,horse,jockey,trainer,class,track,location,actual_weight,draw,distance_m,win_odds,date
0,2020/2021,1,1,DEMONS ROCK,C L Chau,D J Hall,Class 5,ALL WEATHER TRACK - 1200m,Sha Tin,133,2,Goo,4.3,04/11/2020
1,2020/2021,1,2,VIVA COUNCIL,H T Mo,L Ho,Class 5,ALL WEATHER TRACK - 1200m,Sha Tin,133,7,Goo,7.4,04/11/2020
2,2020/2021,1,3,WELL DRAGON,T Piccone,W Y So,Class 5,ALL WEATHER TRACK - 1200m,Sha Tin,133,9,Goo,11,04/11/2020
3,2020/2021,1,4,ADONIS,A Badel,D J Whyte,Class 5,ALL WEATHER TRACK - 1200m,Sha Tin,132,6,Goo,9.3,04/11/2020
4,2020/2021,1,5,TAILOR MADE,Z Purton,C W Chang,Class 5,ALL WEATHER TRACK - 1200m,Sha Tin,132,4,Goo,6.1,04/11/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,2020/2021,8,8,GLORIOUS ARTIST,K C Leung,F C Lor,Class 2,ALL WEATHER TRACK - 1650m,Sha Tin,116,11,Goo,19,04/11/2020
8,2020/2021,8,9,ENRICHMENT,M F Poon,J Size,Class 2,ALL WEATHER TRACK - 1650m,Sha Tin,114,8,Goo,20,04/11/2020
9,2020/2021,8,10,ARROGANT,M L Yeung,C Fownes,Class 2,ALL WEATHER TRACK - 1650m,Sha Tin,113,12,Goo,59,04/11/2020
10,2020/2021,8,11,STAR PERFORMANCE,M Chadwick,A S Cruz,Class 2,ALL WEATHER TRACK - 1650m,Sha Tin,113,5,Goo,12,04/11/2020


In [ ]:
df_race

In [10]:
back_up='../data/latest/coming_race_'+str(next_race_day.year)+str(next_race_day.month)+str(next_race_day.day)+'.csv'
df_race.to_csv('../data/latest/coming_race.csv',index=False)
df_race.to_csv(back_up,index=False)